**Fine tune a VITS model with the Coqui TTS framework using audio samples of your choice.**
R4

Thank you to all of the [Coqui TTS](https://https://github.com/coqui-ai/TTS) contributors, the OpenAI team,and [@Thorsten-Voice](https://https://www.youtube.com/c/ThorstenMueller ) (I think he wrote this original script). I have no idea who wrote the rnnoise loop, but they are great, too. I just smash things together. [-nn](https://https://www.youtube.com/c/NanoNomad )

If you have run the script before, and want to examine your training session using Tensorboard, set your dataset directories, then skip to the bottom section to load Tensorboard.

**Run this cell to connect your Google Drive account to save files.**

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

**Set paths and then run the next cell**

ds_name is the dataset directory (will be created)

output_directory is training storage directory,

subdirectory of ds_name (will be created)

upload_dir is where your samples are stored (will be created)

MODEL_FILE is the default path to the VITS model downloaded using Coqui (do not need to change)

RUN_NAME is a short name describing your training run

In [ ]:
import os

ds_name = "vits-rod-bettersplit-ds" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "vits-sampleuploads" #@param {type:"string"}
MODEL_FILE = "/root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
RUN_NAME = "VITS-en" #@param {type:"string"}


OUT_PATH = "/content/drive/MyDrive/"+ds_name+"/traineroutput/"
!mkdir $upload_dir
!mkdir /content/drive/MyDrive/$ds_name
!mkdir /content/drive/MyDrive/$ds_name/wavs/

mkdir: cannot create directory ‘/content/drive/MyDrive/vits-sampleuploads’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/vits-rod-bettersplit-ds’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/vits-rod-bettersplit-ds/wavs/’: File exists


**Set run type.**

Continue to resume an interrupted session

restore to begin a new session from the defalt model model file above (download from Coqui Hub using the download cell later on).

restore-ckpt is for beginning a new session using a prior fine-tuned checkpoint. You can set this later on in the training section in Part 2.

In [ ]:
run_type = "restore" #@param ["continue","restore","restore-ckpt"]
print(run_type + " run selected")

restore run selected


**Download and Build Rnnoise (https://github.com/xiph/rnnoise) and Requirements**

In [ ]:
#@title
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'rnnoise'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 420 (delta 194), reused 192 (delta 192), pack-reused 201
Receiving objects: 100% (420/420), 836.04 KiB | 12.86 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
autoconf is already the newest version (2.69-11.1).
autoconf set to manually installed.
automake is already the newest version (1:1.16.1-4ubuntu6).
automake set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu4).
curl is already the newest version (7.68.0-1ubuntu2.15).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following package was automatically 

**Install Sox, Install OpenAI Whisper STT+Translation (https://github.com/openai/whisper)**

In [ ]:
#@title
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git


/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Cloning into 'whisper'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 479 (delta 83), reused 111 (delta 69), pack-reused 343
Receiving objects: 100% (479/479), 7.50 MiB | 15.87 MiB/s, done.
Resolving deltas: 100% (282/282), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-22gwynh0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-22gwyn

**Install Coqui TTS** (https://github.com/coqui-ai/TTS), espeak-ng phonemeizer (https://github.com/espeak-ng/espeak-ng), download Coqui TTS source and examples from GitHub.
**bold text**
Currently set to force install Coqui Trainer==0.0.20.  0.0.22 is out, but I have not tested it.  All of my training runs with 0.0.21 failed.

In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
!pip install Trainer==0.0.20

**(Optional) List pretrained models available on the Coqui Hub**

In [ ]:
!tts --list_models

 Name format: type/language/dataset/model
 1: tts_models/multilingual/multi-dataset/your_tts
 2: tts_models/bg/cv/vits
 3: tts_models/cs/cv/vits
 4: tts_models/da/cv/vits
 5: tts_models/et/cv/vits
 6: tts_models/ga/cv/vits
 7: tts_models/en/ek1/tacotron2
 8: tts_models/en/ljspeech/tacotron2-DDC
 9: tts_models/en/ljspeech/tacotron2-DDC_ph
 10: tts_models/en/ljspeech/glow-tts
 11: tts_models/en/ljspeech/speedy-speech
 12: tts_models/en/ljspeech/tacotron2-DCA
 13: tts_models/en/ljspeech/vits
 14: tts_models/en/ljspeech/vits--neon
 15: tts_models/en/ljspeech/fast_pitch
 16: tts_models/en/ljspeech/overflow
 17: tts_models/en/vctk/vits
 18: tts_models/en/vctk/fast_pitch
 19: tts_models/en/sam/tacotron-DDC
 20: tts_models/en/blizzard2013/capacitron-t2-c50
 21: tts_models/en/blizzard2013/capacitron-t2-c150_v2
 22: tts_models/es/mai/tacotron2-DDC
 23: tts_models/es/css10/vits
 24: tts_models/fr/mai/tacotron2-DDC
 25: tts_models/fr/css10/vits
 26: tts_models/uk/mai/glow-tts
 27: tts_models/uk/ma

**Audio Preprocessing Options**

Recommended: Leave all 'true'

run_denoise use Xiph's rnnoise on samples

run_splits split samples based on silence interval of 0.2 seconds and then force a split into 8 second segments. Click view code and find the 'sox' lines to change these intervals.

use_audio_filter engage highpass filter 50hz, lowpass fitler 8000hz. Click view code and find the 'sox' lines to change these frequencies if needed.

normalize_audio to engage -6db peak -25LUFS normalization

In [ ]:
run_denoise = "True" #@param ["True", "False"]
run_splits = "True" #@param ["True", "False"]
use_audio_filter = "True" #@param ["True", "False"]
normalize_audio = "True" #@param ["True", "False"]
#start_sil_dur = 0.2 #@param {type:"number"}
#end_sil_dur = 0.2 #@param {type:"number"}
#sample_max = 8 #@param {type:"number"}


**Process**
This section will convert mp3 and wav files in upload_dir to  22050hz mono wav files.  Then it will pass the wav files through rnnoise.

rnnoise output is then segmented based on 0.2 second silences (click show code below, change 0.2 in the sox line to the duration to silence duration if needed)

8000hz Highpass and 50hz lowpass filters applied, gain/loudness adjusted to reduce potential clipping, -6db peak normalization and -25db lufs applied.  Should be fine for general purpose.

segmented audio is then passed through sox again to force-split any long segments (above 8 seconds) into segments once again.  Files smaller than 35kb are deleted.

In [ ]:
#@title

from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
%cd $upload_dir
#!ls -al
!rm -rf $upload_dir/22k_1ch
!mkdir $upload_dir/22k_1ch

#Convert and resample uploaded mp3/wav clips to 1 channel, 22khz
!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 22k_1ch/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 22k_1ch/"${f%.wav}".wav ; done' _ {} +
#!ls -al $upload_dir/22k_1ch
print("Files converted to 22khz 1ch wav")
if run_denoise=="True":
  print("Running denoise...")
  orig_wavs= upload_dir + "/22k_1ch/"
  print(orig_wavs)

  from pathlib import Path
  import os
  import subprocess
  import soundfile as sf
  import pyloudnorm as pyln
  import sys
  import glob
  rnn = "/content/rnnoise/examples/rnnoise_demo"
  paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
  for filepath in paths:
    base = os.path.basename(filepath)
    tp_s = upload_dir + "/22k_1ch/denoise/"
    tf_s = upload_dir + "/22k_1ch/denoise/" + base
    target_path = Path(tp_s)
    target_file = Path(tf_s)
    print("From: " + str(filepath))
    print("To: " + str(target_file))

  # Stereo to Mono; upsample to 48000Hz
  # added -G to fix gain, -v 0.8
    subprocess.run(["sox", "-G", "-v", "0.8", filepath, "48k.wav", "remix", "-", "rate", "48000"])
    subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
    subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
    subprocess.run(["sox", "-G", "-v", "0.8", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

    subprocess.run(["mkdir", "-p", str(target_path)])
    if use_audio_filter=="True":
      print("Running highpass/lowpass & resample")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "50", "lowpass", "8000", "rate", "22050"])
      # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
    elif use_audio_filter=="False":
      print("Running resample without filter")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "rate", "22050"])
      # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
# peak normalize audio to -6 dB
    if normalize_audio=="True":
      print("Output normalized")
      peak_normalized_audio = pyln.normalize.peak(data, -6.0)

# measure the loudness first
      meter = pyln.Meter(rate) # create BS.1770 meter
      loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
      loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
      sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
      print("")
    elif normalize_audio=="False":
      print("File written without normalizing")
      sf.write(target_file, data=data, samplerate=22050)
      print("")

  !rm $target_path/rnn.wav
  !rm $target_path/48k.wav

elif run_denoise=="False":
  paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
  for filepath in paths:
    print("Skipping denoise...")
    base = os.path.basename(filepath)
    tp_s = upload_dir + "/22k_1ch/denoise/"
    tf_s = upload_dir + "/22k_1ch/denoise/" + base
    target_path = Path(tp_s)
    target_file = Path(tf_s)
    print("From: " + str(filepath))
    print("To: " + str(target_file))
    subprocess.run(["sox", "-G", "-v", "0.8", filepath, "48k.wav", "remix", "-", "rate", "48000"])
    subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
    #subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
    subprocess.run(["sox", "-G", "-v", "0.8", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav
    subprocess.run(["mkdir", "-p", str(target_path)])
    if use_audio_filter=="True":
      print("Running filter...")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "50", "lowpass", "8000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
    elif use_audio_filter=="False":
      print("Skipping filter...")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
          # peak normalize audio to -6 dB
    if normalize_audio=="True":
      print("Output normalized")
      peak_normalized_audio = pyln.normalize.peak(data, -6.0)

# measure the loudness first
      meter = pyln.Meter(rate) # create BS.1770 meter
      loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
      loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
      sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
      print("")
    if normalize_audio=="False":
      print("File written without normalizing")
      sf.write(target_file, data=data, samplerate=22050)
      print("")
  !rm $target_path/rnn.wav
  !rm $target_path/48k.wav

if run_splits=="False":
  print("Copying files without splitting...")
  %mkdir /content/drive/MyDrive/$ds_name
  %mkdir /content/drive/MyDrive/$ds_name/wavs
  !cp $target_path/*.wav /content/drive/MyDrive/$ds_name/wavs
if run_splits=="True":
  print("Splitting output and copying...")
  %cd $target_path
  !rm -rf splits
  !mkdir splits
  !for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.2 0.1% 1 0.2 0.1% : newfile : restart ; done
#alt split method: force splits of 8 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
  %cd splits
  !mkdir resplit
  !for FILE in *.wav; do sox "$FILE" resplit/"$FILE" --show-progress trim 0 8 : newfile : restart ; done
  %cd resplit
  !find . -name "*.wav" -type f -size -35k -delete
  #!ls -al
  %cd /content/drive/MyDrive/$ds_name/

  !mkdir wavs
  !cp $target_path/splits/resplit/*.wav /content/drive/MyDrive/$ds_name/wavs
  %cd /content/drive/MyDrive/$ds_name/wavs
#  !ls -al

/content/drive/MyDrive/vits-sampleuploads
Files converted to 22khz 1ch wav
Running denoise...
/content/drive/MyDrive/vits-sampleuploads/22k_1ch/
From: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/rsow-06.wav
To: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/denoise/rsow-06.wav
Running highpass/lowpass & resample
Output normalized

From: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/rsow-09.wav
To: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/denoise/rsow-09.wav
Running highpass/lowpass & resample
Output normalized

From: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/rsow-02.wav
To: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/denoise/rsow-02.wav
Running highpass/lowpass & resample
Output normalized

From: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/rsow-13.wav
To: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/denoise/rsow-13.wav
Running highpass/lowpass & resample
Output normalized

From: /content/drive/MyDrive/vits-sampleuploads/22k_1ch/rsow-10.wav

/usr/local/lib/python3.8/dist-packages/pyloudnorm/normalize.py:58: RuntimeWarning: invalid value encountered in multiply
  output = gain * data


Running highpass/lowpass & resample
Output normalized



/usr/local/lib/python3.8/dist-packages/pyloudnorm/normalize.py:58: RuntimeWarning: invalid value encountered in multiply
  output = gain * data


Streaming output truncated to the last 5000 lines.
Sample Encoding: 16-bit Signed Integer PCM

In:100%  00:00:05.32 [00:00:00.00] Out:116k  [ =====|===== ]        Clip:0    
Done.

Input File     : 'rsow-187.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:01.35 = 29767 samples ~ 101.248 CDDA sectors
File Size      : 59.6k
Bit Rate       : 353k
Sample Encoding: 16-bit Signed Integer PCM

In:100%  00:00:01.35 [00:00:00.00] Out:28.6k [ -====|====- ]        Clip:0    
Done.

Input File     : 'rsow-188.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:04.34 = 95697 samples ~ 325.5 CDDA sectors
File Size      : 191k
Bit Rate       : 353k
Sample Encoding: 16-bit Signed Integer PCM

In:100%  00:00:04.34 [00:00:00.00] Out:95.4k [-=====|=====-]        Clip:0    
Done.

Input File     : 'rsow-189.wav'
Channels       : 1
Sample Rate    : 22050
Precision      : 16-bit
Duration       : 00:00:01.45 = 31972 sampl

**Run this cell once only.**

**Load OpenAI Whisper large-v2  STT model to memory.  Loading it multiple times tends to crash Colab.**

Click show code and swap the commented line to use the medium.en model instead

In [ ]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path


#model = whisper.load_model("medium.en")
model = whisper.load_model("large-v2")

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 108MiB/s]


**Run Whisper on generated audio clips, generate transcript named metadata.csv in LJSpeech format in the dataset directory.**

In [ ]:
#@title
wavs = '/content/drive/MyDrive/'+ds_name+'/wavs'

paths = glob.glob(os.path.join(wavs, '*.wav'))
print(len(paths))

all_filenames = []
transcript_text = []
with open('/content/drive/MyDrive/'+ds_name+'/metadata.csv', 'w', encoding='utf-8') as outfile:
	for filepath in paths:
		base = os.path.basename(filepath)
		all_filenames.append(base)
	for filepath in paths:
		result = model.transcribe(filepath)
		output = result["text"].lstrip()
		output = output.replace("\n","")
		thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
		outfile.write(thefile + '|' + output + '|' + output + '\n')
		print(thefile + '|' + output + '|' + output + '\n')


236
rsow-01001001|They come from every human experience that you either witness or have heard about.|They come from every human experience that you either witness or have heard about.

rsow-02001001|translated into your brain in your own sense of dialogue, in your own language form.|translated into your brain in your own sense of dialogue, in your own language form.

rsow-03002001|ideas are born.|ideas are born.

rsow-04002001|from what is smelled.|from what is smelled.

rsow-05002001|Seen, experienced, felt, emotionalized.|Seen, experienced, felt, emotionalized.

rsow-06001001|ideas are probably...|ideas are probably...

rsow-07001001|in the air.|in the air.

rsow-07002001|like little tiny items of ozone.|like little tiny items of ozone.

rsow-08001001|That's the easiest thing on Earth, is to come up with an idea.|That's the easiest thing on Earth, is to come up with an idea.

rsow-09001001|And the second thing is, the hardest thing on earth is to put it down.|And the second thing is,

**Display dataset**

In [ ]:
#@title
!cat /content/drive/MyDrive/$ds_name/metadata.csv

**Download VITS model and Generate Sample Wav File to /content/ljspeech-vits.wav  This will be deleted when your Colab session is closed.**

In [ ]:
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--vits
100% 146M/146M [00:18<00:00, 8.03MiB/s]
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: I a

**Load Tensorboard**

In [ ]:
import torch
%load_ext tensorboard

**Load Dashboard**
May take several minutes to appear from a blank white box.  Ad blockers probably need to whitelist a bunch of Colab stuff or this won't work.

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$ds_name/$output_directory/

**Load libs**

In [ ]:
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor


In [ ]:
output_path = "/content/drive/MyDrive/"+ds_name + "/" + output_directory + "/"
SKIP_TRAIN_EPOCH = False

In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "/content/drive/MyDrive/"+ ds_name)
)

In [ ]:
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=16,
    eval_batch_size=16,
    batch_group_size=16,
#    num_loader_workers=8,
    num_loader_workers=2,
    num_eval_loader_workers=2,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100000,
    save_step=1000,
	  save_checkpoints=True,
	  save_n_checkpoints=4,
	  save_best_after=1000,
    #text_cleaner="english_cleaners",
    text_cleaner="multilingual_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:
# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 236 files in /content/drive/MyDrive/vits-rod-bettersplit-ds


In [ ]:
model = Vits.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


**If continuning a run: use the next cell to list all run directories.**

**Copy and paste the run you want to or restore a checkpoint from into the next box**

In [ ]:
#@title
!ls -al /content/drive/MyDrive/$ds_name/traineroutput

ls: cannot access '/content/drive/MyDrive/vits-rod-bettersplit-ds/traineroutput': No such file or directory


**Run folder to continue from or Run folder that contains your restore checkpoint**

In [ ]:
run_folder = "vits_ljspeech-February-01-2023_01+20AM-0000000" #@param {type:"string"}


List checkpoints in run folder. The checkpoint only needs to be selected for a restore run.

Continuing a run will load the last best loss checkpoint according to the stored config.json in the run directory on its own (a directory is specified for a continue run, and a model file is specified for a restore run)

In [ ]:
#@title
!ls -al /content/drive/MyDrive/$ds_name/traineroutput/$run_folder

total 6034391
-rw------- 1 root root 997870789 Feb  1 03:32 best_model_1006587.pth
-rw------- 1 root root 997870789 Feb  1 04:37 best_model.pth
-rw------- 1 root root 997870789 Feb  1 07:20 checkpoint_1019000.pth
-rw------- 1 root root 997870789 Feb  1 07:39 checkpoint_1020000.pth
-rw------- 1 root root 997870789 Feb  1 07:58 checkpoint_1021000.pth
-rw------- 1 root root 997870789 Feb  1 08:17 checkpoint_1022000.pth
-rw------- 1 root root      8151 Feb  1 01:20 config.json
-rw------- 1 root root 190700409 Feb  1 08:33 events.out.tfevents.1675214454.2d45acdd69ab
-rw------- 1 root root       451 Feb  1 01:20 ipykernel_launcher.py
-rw------- 1 root root   1279836 Feb  1 10:35 trainer_0_log.txt


**If changing to a different "restore" checkpoint to begin a new training session with a model you are already training, set the checkpoint filename here**

In [ ]:
ckpt_file = "checkpoint_1022000.pth" #@param {type:"string"}
print(ckpt_file + " selected for restore run")
if run_type=="continue":
  print("Warning:\n restore checkpoint selected, but run type set to continue.\nTrainer will load best loss from checkpoint directory.\n Are you sure this is what you want to do?\n\nIf not, change the run type below to 'restore'")
elif run_type=="restore-ckpt":
  print("Warning:\n restore checkpoint selected, run type set to restore from selected checkpoint, not default base model.\nIf this is not correct, adjust the run type.")


checkpoint_1022000.pth selected for restore run
 restore checkpoint selected, run type set to restore from selected checkpoint, not default base model.
If this is not correct, adjust the run type.


**Last chance to change run type**

In [ ]:
run_type = "restore" #@param ["continue","restore","restore-ckpt"]
print(run_type + " run selected")

restore run selected


**(Session recovery: Reset selected model file back to default predownloaded path)**

In [ ]:
#@title
ckpt_file = "/root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth"
print(ckpt_file + " selected for restore run")

/root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth selected for restore run


**(Optional) Freeze selected modules. Trainer must be reinitilized if these are changed.**

In [ ]:
print("Current reinit_text_encoder value: " + str(config.model_args.reinit_text_encoder))
reinit_te_status = "False" #@param ["False", "True"]
if reinit_te_status=="False":
  print("Text encoder will not be reinitilized")
elif reinit_te_status=="True":
  config.model_args.reinit_text_encoder=True
  print("Model arguments set to reinitilize text encoder")
  print("Current reinit_DP value: " + str(config.model_args.reinit_DP))
reinit_DP_status = "False" #@param ["False", "True"]
if reinit_DP_status=="False":
  print("DP will not be reinitilized")
elif reinit_DP_status=="True":
  config.model_args.reinit_DP=True
  print("Model arguments set to reinitilize DP")
print("Current freeze_waveform_decoder value: " + str(config.model_args.freeze_waveform_decoder))
freeze_waveform_decoder_status = "False" #@param ["False", "True"]
if freeze_waveform_decoder_status=="False":
  print("Waveform decoder will NOT be frozen")
  config.model_args.freeze_waveform_decoder=False
elif freeze_waveform_decoder_status=="True":
  config.model_args.freeze_waveform_decoder=True
  print("Waveform decoder FROZEN")
print("Current freeze_flow_decoder value: " + str(config.model_args.freeze_flow_decoder))
freeze_flow_decoder_status = "False" #@param ["False", "True"]
if freeze_flow_decoder_status=="False":
  print("Flow decoder will NOT be frozen")
  config.model_args.freeze_flow_decoder=None
elif freeze_flow_decoder_status=="True":
  config.model_args.freeze_flow_decoder="True"
  print("Flow decoder FROZEN")
print("Current freeze_encoder value: " + str(config.model_args.freeze_encoder))
freeze_encoder_status = "False" #@param ["False", "True"]
if freeze_encoder_status=="False":
  print("Text encoder will NOT be frozen")
  config.model_args.freeze_encoder=False
elif freeze_encoder_status=="True":
  config.model_args.freeze_encoder=True
  print("Text encoder FROZEN")
print("Current freeze_DP value: " + str(config.model_args.freeze_DP))
freeze_DP_status = "False" #@param ["False", "True"]
if freeze_DP_status=="False":
  print("Duration predictor will NOT be frozen")
  config.model_args.freeze_DP=False
elif freeze_DP_status=="True":
  config.model_args.freeze_DP=True
  print("Duration predictor FROZEN")

Current reinit_text_encoder value: False
Model arguments set to reinitilize text encoder
Current reinit_DP value: False
DP will not be reinitilized
Current freeze_waveform_decoder value: False
Waveform decoder will NOT be frozen
Current freeze_flow_decoder value: False
Flow decoder will NOT be frozen
Current freeze_encoder value: False
Text encoder will NOT be frozen
Current freeze_DP value: False
Duration predictor will NOT be frozen


**Init the trainer**

In [ ]:
#@title
print(run_type)
if run_type=="continue":
  CONTINUE_PATH="/content/drive/MyDrive/"+ds_name+"/traineroutput/"+run_folder
  trainer = Trainer(
    TrainerArgs(continue_path=CONTINUE_PATH, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
elif run_type=="restore":
    trainer = Trainer(
    TrainerArgs(restore_path=MODEL_FILE, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
elif run_type=="restore-ckpt":
  trainer = Trainer(
  TrainerArgs(restore_path="/content/drive/MyDrive/"+ds_name+"/traineroutput/"+run_folder+"/"+ckpt_file, skip_train_epoch=SKIP_TRAIN_EPOCH),
  config,
  output_path=OUT_PATH,
  model=model,
  train_samples=train_samples,
  eval_samples=eval_samples,
)

restore


 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/vits-rod-bettersplit-ds/traineroutput/vits_ljspeech-February-03-2023_08+27PM-0000000
 > Restoring from model_file.pth ...
 > Restoring Model...
 > Partial model initialization...
 | > Layer missing in the checkpoint: disc.nets.0.convs.0.bias
 | > Layer missing in the checkpoint: disc.nets.0.convs.0.weight_g
 | > Layer missing in the checkpoint: disc.nets.0.convs.0.weight_v
 | > Layer missing in the checkpoint: disc.nets.0.convs.1.bias
 | > Layer missing in the checkpoint: disc.nets.0.convs.1.weight_g
 | > Layer missing in the checkpoint: disc.nets.0.convs.1.weight_v
 | > Layer missing in the checkpoint: disc.nets.0.convs.2.bias
 | > Layer missing in the checkpoint: disc.nets.0.convs.2.we

 > Text Encoder was reinit.


**Run training**

In [ ]:
trainer.fit()


 > EPOCH: 0/100000
 --> /content/drive/MyDrive/vits-rod-bettersplit-ds/traineroutput/vits_ljspeech-February-03-2023_08+27PM-0000000


[*] Pre-computing phonemes...


  1%|          | 2/234 [00:00<00:55,  4.15it/s]

['<BLNK>', 's', '<BLNK>', 'ˈ', '<BLNK>', 'ɜ', '<BLNK>', 'ː', '<BLNK>', 'ʔ', '<BLNK>', 'n', '<BLNK>', '̩', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 'p', '<BLNK>', 'ˈ', '<BLNK>', 'ɛ', '<BLNK>', 'ʃ', '<BLNK>', 'ə', '<BLNK>', 'l', '<BLNK>', ' ', '<BLNK>', 'h', '<BLNK>', 'ˈ', '<BLNK>', 'æ', '<BLNK>', 'ŋ', '<BLNK>', ' ', '<BLNK>', 'ˈ', '<BLNK>', 'ʌ', '<BLNK>', 'p', '<BLNK>', 's', '<BLNK>', ',', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 'ˈ', '<BLNK>', 'ɜ', '<BLNK>', 'ː', '<BLNK>', 'ʔ', '<BLNK>', 'n', '<BLNK>', '̩', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 'p', '<BLNK>', 'ˈ', '<BLNK>', 'ɛ', '<BLNK>', 'ʃ', '<BLNK>', 'ə', '<BLNK>', 'l', '<BLNK>', ' ', '<BLNK>', 'p', '<BLNK>', 'ɹ', '<BLNK>', 'ɪ', '<BLNK>', 'ˌ', '<BLNK>', 'ɑ', '<BLNK>', 'ː', '<BLNK>', 'k', '<BLNK>', 'j', '<BLNK>', 'ʊ', '<BLNK>', 'p', '<BLNK>', 'ˈ', '<BLNK>', 'e', '<BLNK>', 'ɪ', '<BLNK>', 'ʃ', '<BLNK>', 'ə', '<BLNK>', 'n', '<BLNK>', 'z', '<BLNK>', ' ', '<BLNK>', 'æ', '<BLNK>', 'n', '<BLNK>', 'd', '<BLNK>', ' ', '<BLNK>', 'p', '<BLNK>',

100%|██████████| 234/234 [01:07<00:00,  3.48it/s]

 > TRAINING (2023-02-03 20:29:13) 


...



> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: espeak
	| > 1 not found characters:
	| > ̩
| > Number of instances : 234
 | > Preprocessing samples
 | > Max text length: 184
 | > Min text length: 8
 | > Avg text length: 49.43589743589744
 | 
 | > Max audio length: 176422.0
 | > Min audio length: 18135.0
 | > Avg audio length: 56915.14957264957
 | > Num. instances discarded samples: 0
 | > Batch group size: 256.



 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: espeak
	| > 1 not found characters:
	| > ̩
| > Number of instances : 2
 | > Preprocessing samples
 | > Max text length: 80
 | > Min text length: 30
 | > Avg text length: 55.0
 | 
 | > Max audio length: 69758.0
 | > Min audio length: 33517.0
 | > Avg audio length: 51637.5
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


   --> STEP: 0
     | > loss_disc: 2.88023  (2.88023)
     | > loss_disc_real_0: 0.31364  (0.31364)
     | > loss_disc_real_1: 0.15978  (0.15978)
     | > loss_disc_real_2: 0.24330  (0.24330)
     | > loss_disc_real_3: 0.24986  (0.24986)
     | > loss_disc_real_4: 0.24475  (0.24475)
     | > loss_disc_real_5: 0.33701  (0.33701)
     | > loss_0: 2.88023  (2.88023)
     | > loss_gen: 1.73790  (1.73790)
     | > loss_kl: 3.19288  (3.19288)
     | > loss_feat: 1.02346  (1.02346)
     | > loss_mel: 17.83686  (17.83686)
     | > loss_duration: 1.21826  (1.21826)
     | > loss_1: 25.00935  (25.00935)



 | > Synthesizing test sentences.


In [ ]:
!nvidia-smi